## EncoderDecoder
----
- 1 如果我们现在要做个中英文翻译,比如我是中国人翻译成 ‘i am Chinese’.这时候我们会发现输入有 5个中文字,而输出只有三个英文单词. 也就是输入长度并不等于输出长度.这时候我们会引入一种 编码器-解码器的模型也就是 (Encoder-Decoder).首先我们通过编码器 对输入 ‘我是中国人’ 进行信息编码, 之后将生成的编码数据输入 decoder 进行解码.一般编码器和解码器 都会使用循环神经网络.
- 2 当然为了使机器知道句子的结束我们会在每个句子后面增加 一个$<eos>$表示句子的结束.使得电脑可以进行识别.在训练的时候 我们也一般会在解码器的第一个输入阶段加上$<bos>$表示预测的开始.
    
- 3 同时为了使每个句子保持相同长度,我们 会人为预先规定句子长度,若句子没有达到长度,那么我们会对句子进行填充,使得其长度达到规定长度.
- 4 作为编码器的输入 我们一般使用 C = q(h1, h2…ht)作为第一个隐藏层输入，一般的我们也可以直接使用c = ht，不用包含之前所有的隐藏层信息。
- 5 在训练的时候我们一般会使用强制教学，也就是 不把y_hat1的预测数据当做编码器的第二个输入， 而是直接用标签数据的y1当做输入。
- 6 当我们使用贪心算法再对y进行softmax的时候，我们对每个输出的y进行当前最优的选择。可能会达到全局最优的情况


In [1]:
import collections
import os
import io
import math
import time
import torch
from torch import nn
import torch.nn.functional as F
import torchtext.vocab as Vocab
import torch.utils.data as Data

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
DATA_ROOT = "./data"
TRIAN_DIR = os.path.join(DATA_ROOT, "train")
if not os.path.exists(TRIAN_DIR):
    os.makedirs(TRIAN_DIR)
    
PAD, UNK, BOS, EOS  = "<pad>", "<unk>", "<bos>", "<eos>"
N_PAD, N_UNK, N_BOS, N_EOS = 0, 1, 2, 3

ZH_VOCAB_SIZE = 4000
ZH_VOCAB_OUTPUT = os.path.join(TRIAN_DIR, "zh.vocab")
ZH_DEV = os.path.join(TRIAN_DIR, "zh.dev")
ZH_TRAIN = os.path.join(TRIAN_DIR, "zh.train")

EN_VOCAB_SIZE = 10000
EN_VOCAB_OUTPUT = os.path.join(TRIAN_DIR, "en.vocab")
EN_DEV = os.path.join(TRIAN_DIR, "en.dev")   # 这个暂时没用？
EN_TRAIN = os.path.join(TRIAN_DIR, "en.train")

In [3]:
def load_data(filename):
    data = []
    with open(filename, "r") as f:
        for line in f:
            sent = line.strip()
            if sent:
                seq = [int(i) for i in sent.split()]
                data.append(seq)
    return data

class Tokenizer(object):
    """解析器"""
    def __init__(self, vocab_file):
        self.vocab_file = vocab_file
        self.vocab_list = self.load_vocab()
        self.word2idx = self.build_word2idx()
        self.idx2word = self.bulid_idx2word()
        
    def load_vocab(self):
        vocab = []
        with open(self.vocab_file, "r") as f:
            for word in f:
                vocab.append(word.strip())
        return vocab
    
    def build_word2idx(self):
        word2idx = {w:i for i, w in enumerate(self.vocab_list)}
        return word2idx
    
    def bulid_idx2word(self):
        idx2word = {i:w for i, w in enumerate(self.vocab_list)}
        return idx2word
    
    def wtoi(self, word_list):
        idx_list = []
        for word in word_list:
            if word not in self.word2idx:
                idx = self.word2idx.get(UNK)
            else:
                idx = self.word2idx.get(word)
            idx_list.append(idx)
        return idx_list
        
    def itow(self, idx_list):
        word_list = []
        for idx in idx_list:
            word = self.idx2word.get(idx)
            word_list.append(word)
        return word_list

In [4]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 drop_prob=0, **kwargs):
        super(Encoder, self).__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.GRU(embed_size, num_hiddens, num_layers, dropout=drop_prob)

    def forward(self, inputs, state):
        # 输入形状是(批量大小, 时间步数)。将输出互换样本维和时间步维
        embedding = self.embedding(inputs.long()).permute(1, 0, 2) # (seq_len, batch, input_size)
        return self.rnn(embedding, state)

    def begin_state(self):
        return None # 隐藏态初始化为None时PyTorch会自动初始化为0

    
def attention_model(input_size, attention_size):
    model = nn.Sequential(nn.Linear(input_size, 
                                    attention_size, bias=False),
                          nn.Tanh(),
                          nn.Linear(attention_size, 1, bias=False))
    return model


def attention_forward(model, enc_states, dec_state):
    """
    enc_states: (时间步数, 批量大小, 隐藏单元个数)
    dec_state: (批量大小, 隐藏单元个数)
    """
    # 将解码器隐藏状态广播到和编码器隐藏状态形状相同后进行连结
    dec_states = dec_state.unsqueeze(dim=0).expand_as(enc_states)
    enc_and_dec_states = torch.cat((enc_states, dec_states), dim=2)
    e = model(enc_and_dec_states)  # 形状为(时间步数, 批量大小, 1)
    alpha = F.softmax(e, dim=0)  # 在时间步维度做softmax运算
    return (alpha * enc_states).sum(dim=0)  # 返回背景变量

In [7]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 attention_size, drop_prob=0):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.attention = attention_model(2*num_hiddens, attention_size)
        # GRU的输入包含attention输出的c和实际输入, 所以尺寸是 num_hiddens+embed_size
        self.rnn = nn.GRU(num_hiddens + embed_size, num_hiddens, 
                          num_layers, dropout=drop_prob)
        self.out = nn.Linear(num_hiddens, vocab_size)

    def forward(self, cur_input, state, enc_states):
        """
        cur_input shape: (batch, )
        state shape: (num_layers, batch, num_hiddens)
        """
        # 使用注意力机制计算背景向量
        c = attention_forward(self.attention, enc_states, state[-1])
        # 将嵌入后的输入和背景向量在特征维连结, (批量大小, num_hiddens+embed_size)
        input_and_c = torch.cat((self.embedding(cur_input), c), dim=1)
        # 为输入和背景向量的连结增加时间步维，时间步个数为1
        output, state = self.rnn(input_and_c.unsqueeze(0), state)
        # 移除时间步维，输出形状为(批量大小, 输出词典大小)
        output = self.out(output).squeeze(dim=0)
        return output, state

    def begin_state(self, enc_state):
        # 直接将编码器最终时间步的隐藏状态作为解码器的初始隐藏状态
        return enc_state
    
    
def batch_loss(encoder, decoder, X, Y, loss):
    batch_size = X.shape[0]
    enc_state = encoder.begin_state()
    enc_outputs, enc_state = encoder(X, enc_state)
    # 初始化解码器的隐藏状态
    dec_state = decoder.begin_state(enc_state)
    
    # 解码器在最初时间步的输入是BOS [out_vocab.stoi[BOS]]
    dec_input = torch.tensor([N_BOS] * batch_size).to(device)
    # 我们将使用掩码变量mask来忽略掉标签为填充项PAD的损失
    mask, num_not_pad_tokens = torch.ones(batch_size,).to(device), 0
    
    l = torch.tensor([0.0]).to(device)
    
    out_vocab_eos = torch.tensor([N_BOS]).to(device)
    for y in Y.permute(1,0): # Y shape: (batch, seq_len)
        dec_output, dec_state = decoder(dec_input, dec_state, enc_outputs)
#         print("dec_output:", dec_output, dec_output.size())
#         print("dec_state:", dec_state, dec_state.size())
#         print("y", y)
#         print("CrossEntropyLoss:", loss(dec_output, y))
#         print("mask", mask)
#         print("-"*30)
        l = l + (mask * loss(dec_output, y)).sum()
        dec_input = y  # 使用强制教学
        num_not_pad_tokens += mask.sum().item()
        # EOS后面全是PAD. 下面一行保证一旦遇到EOS接下来的循环中mask就一直是0 out_vocab.stoi[EOS]
        
        mask = mask * (y != out_vocab_eos).float()
    return l / num_not_pad_tokens


def train(encoder, decoder, dataset, lr, batch_size, num_epochs):
    enc_optimizer = torch.optim.Adam(encoder.parameters(), lr=lr)
    dec_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)

    loss = nn.CrossEntropyLoss(reduction='none').to(device)
    data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)
    for epoch in range(num_epochs):
        l_sum = 0.0
        start = time.time()
        n = 0
        for X, Y in data_iter:
            X = X.to(device)
            Y = Y.to(device)
            enc_optimizer.zero_grad()
            dec_optimizer.zero_grad()
            l = batch_loss(encoder, decoder, X, Y, loss)
#             print(l)
            l.backward()
            enc_optimizer.step()
            dec_optimizer.step()
            l_sum += l.item()
            n += 1
            if n % 2000 == 0:
                print("iter -- {}".format(n))
#             break
#         if (epoch + 1) % 10 == 0:
        end = time.time()
    
        print("epoch %d, loss %.3f, time: %.2f" % (epoch + 1, l_sum / len(data_iter), end-start))

In [8]:
en_data = load_data(EN_TRAIN)
zh_data = load_data(ZH_TRAIN)

In [9]:
def max_len(data):
    return max([len(s) for s in data])

def min_len(data):
    return min([len(s) for s in data])

In [10]:
[max_len(en_data), max_len(zh_data)]

[644, 883]

In [11]:
[min_len(en_data), min_len(zh_data)]

[1, 1]

In [12]:
def process_one_seq(src_tokens, trg_tokens, max_seq_len):
    all_tokens = []
    for src, trg in zip(src_tokens, trg_tokens):
        src_len = len(src)
        trg_len = len(trg)
        if src_len > (max_seq_len-1) or trg_len > (max_seq_len-1):
            continue
        else:
            src_tokens = src + [N_EOS] + [N_PAD] * (max_seq_len - src_len - 1)
            trg_tokens = trg + [N_EOS] + [N_PAD] * (max_seq_len - trg_len - 1)
        all_tokens.append((src_tokens, trg_tokens))
    return all_tokens

def build_dataset(all_data):
    in_data = []
    out_data = []
    for seq_tuple in all_data:
        x, y = seq_tuple
        in_data.append(x)
        out_data.append(y)
    in_data = torch.tensor(in_data)
    out_data = torch.tensor(out_data)
    return Data.TensorDataset(in_data, out_data)

In [13]:
seq_len, batch_size, num_hiddens = 128, 64, 256
model = attention_model(2*num_hiddens, seq_len) 
enc_states = torch.zeros((seq_len, batch_size, num_hiddens))
dec_state = torch.zeros((batch_size, num_hiddens))
attention_forward(model, enc_states, dec_state).shape # torch.Size([16, 256])

torch.Size([64, 256])

In [14]:
all_data = process_one_seq(en_data, zh_data, seq_len)

In [15]:
len(all_data)

178738

In [16]:
dataset = build_dataset(all_data)

In [17]:
in_tokenizer = Tokenizer(EN_VOCAB_OUTPUT)
out_tokenizer = Tokenizer(ZH_VOCAB_OUTPUT)
in_vocab_len = len(in_tokenizer.word2idx)
out_vocab_len = len(out_tokenizer.word2idx)

In [18]:
embed_size, num_hiddens, num_layers = 256, 256, 1
attention_size, drop_prob, lr, batch_size, num_epochs = seq_len, 0.5, 0.01, 2, 50
encoder = Encoder(in_vocab_len, embed_size, num_hiddens, num_layers,
                  drop_prob)
encoder = encoder.to(device)
decoder = Decoder(out_vocab_len, embed_size, num_hiddens, num_layers,
                  attention_size, drop_prob)
decoder = decoder.to(device)

/usr/local/anaconda2/envs/pt-tf-env/lib/python3.6/site-packages/torch/nn/modules/rnn.py:54: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
train(encoder, decoder, dataset, lr, batch_size, num_epochs)

iter -- 2000
iter -- 4000
iter -- 6000
iter -- 8000
iter -- 10000
iter -- 12000
iter -- 14000
iter -- 16000
iter -- 18000
iter -- 20000
iter -- 22000
iter -- 24000
iter -- 26000
iter -- 28000
iter -- 30000
iter -- 32000
iter -- 34000
iter -- 36000
iter -- 38000
iter -- 40000
iter -- 42000
iter -- 44000
iter -- 46000
iter -- 48000
iter -- 50000
iter -- 52000
iter -- 54000
iter -- 56000
iter -- 58000
iter -- 60000
iter -- 62000
iter -- 64000
iter -- 66000
iter -- 68000
iter -- 70000
iter -- 72000
iter -- 74000
iter -- 76000
iter -- 78000
iter -- 80000
iter -- 82000
iter -- 84000
iter -- 86000
iter -- 88000
epoch 1, loss nan, time: 12111.99
iter -- 2000
iter -- 4000
iter -- 6000
iter -- 8000
iter -- 10000
iter -- 12000
iter -- 14000
iter -- 16000
iter -- 18000
iter -- 20000
iter -- 22000
iter -- 24000
iter -- 26000
iter -- 28000
iter -- 30000
iter -- 32000
iter -- 34000
iter -- 36000
iter -- 38000
iter -- 40000
iter -- 42000
iter -- 44000
iter -- 46000
iter -- 48000
iter -- 50000
iter -- 